# Second Project OPTIMIZATION MODEL

In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 21.4 MB/s eta 0:00:00


In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from pulp import *

# Load dataset
df = pd.read_csv('/content/Train.csv')  # Make sure 'Train.csv' is in your working directory

# Let's assume the relevant columns are:
# 'Warehouse_block', 'Product_importance', 'Demand', 'Weight_in_gms', 'Cost_of_the_Product'

# Group and clean data (assumption-based)
df = df[['Warehouse_block', 'Product_importance', 'Reached.on.Time_Y.N', 'Weight_in_gms', 'Cost_of_the_Product']].dropna()

# Rename columns for clarity to match the original code's logic
df = df.rename(columns={'Warehouse_block': 'Warehouse',
                        'Product_importance': 'Product Category',
                        'Reached.on.Time_Y.N': 'Demand', # Assuming Reached.on.Time_Y.N can be used as a proxy for demand (1 if reached, 0 if not)
                        'Weight_in_gms': 'Inventory', # Assuming Weight_in_gms can be used as a proxy for inventory
                        'Cost_of_the_Product': 'Shipping Cost'}) # Assuming Cost_of_the_Product can be used as a proxy for shipping cost


# Unique warehouses and product categories
warehouses = df['Warehouse'].unique()
products = df['Product Category'].unique()

# Create a dictionary for shipping cost, inventory, and demand
shipping_costs = {(w, p): df[(df['Warehouse'] == w) & (df['Product Category'] == p)]['Shipping Cost'].values[0]
                  for w in warehouses for p in products if not df[(df['Warehouse'] == w) & (df['Product Category'] == p)].empty}

inventory = {w: df[df['Warehouse'] == w]['Inventory'].sum() for w in warehouses}
demand = {p: df[df['Product Category'] == p]['Demand'].sum() for p in products}

# Initialize LP problem
model = LpProblem("Supply_Chain_Cost_Minimization", LpMinimize)

# Decision variables: quantity shipped from warehouse w to fulfill product p
x = LpVariable.dicts("ship_qty", (warehouses, products), lowBound=0, cat='Continuous')

# Objective: Minimize total shipping cost
model += lpSum([shipping_costs[w, p] * x[w][p] for w in warehouses for p in products if (w, p) in shipping_costs])

# Constraints
# 1. Meet product demand
for p in products:
    model += lpSum([x[w][p] for w in warehouses if (w, p) in shipping_costs]) >= demand[p], f"Demand_{p}"

# 2. Do not exceed warehouse inventory
for w in warehouses:
    model += lpSum([x[w][p] for p in products if (w, p) in shipping_costs]) <= inventory[w], f"Inventory_{w}"

# Solve the model
model.solve()

# Output results
print("Status:", LpStatus[model.status])
print("Total Minimized Cost:", value(model.objective))

# Show shipment plan
for w in warehouses:
    for p in products:
        if (w, p) in shipping_costs:
            shipped_qty = x[w][p].varValue
            if shipped_qty > 0:
                print(f"Ship {shipped_qty:.0f} units of {p} from {w}")

Status: Optimal
Total Minimized Cost: 1028623.0
Ship 2807 units of medium from A
Ship 3140 units of low from C
Ship 616 units of high from C
